In [29]:
# Import Libraries and modules

import pandas as pd
from datetime import datetime
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import os

# **Preparing the Bike_df dataset**

We have 2 datasets to prepare, clean and merge to obtain a dataset presenting the demand for rented bikes and the various weather factors in Marseille since January 2023 (Bike_df). 

In [30]:
dataset = pd.read_csv("C:/Users/39812/OneDrive - Aix Marseille Provence Métropole/Documents/Bike Project/Datasets/Data.csv")
dataset.head()

,Dates,__i18n__01_04_02__column__hour,__i18n__01_04_02__column__type,__i18n__01_04_02__column__trips
0,2022-12-19,0,__i18n__enums__day__Week,2.0
1,2022-12-19,1,__i18n__enums__day__Week,1.0
2,2022-12-19,3,__i18n__enums__day__Week,1.0
3,2022-12-19,6,__i18n__enums__day__Week,1.0
4,2022-12-19,7,__i18n__enums__day__Week,3.0


## This is the first dataset, presenting the demand for bicycles with associated dates, types of day and times.

In [31]:
#renaming

dataset = dataset.rename(columns={
    '__i18n__01_04_02__column__hour' : 'Hour',
    '__i18n__01_04_02__column__type' : 'Weekdays_or_weekend',
    '__i18n__01_04_02__column__trips': 'Rented Bike Count'
})

dataset['Rented Bike Count'] = dataset['Rented Bike Count'].astype(int)

In [32]:
#enconding day type variable 

dataset['Weekdays_or_weekend'] = dataset['Weekdays_or_weekend'].replace({
    '__i18n__enums__day__Week': 1,
    '__i18n__enums__day__Weekend': 0
})
dataset.head()

,Dates,Hour,Weekdays_or_weekend,Rented Bike Count
0,2022-12-19,0,1,2
1,2022-12-19,1,1,1
2,2022-12-19,3,1,1
3,2022-12-19,6,1,1
4,2022-12-19,7,1,3


In [33]:
# Change The datatype of Date columns to extract 'Month' ,'Day', "year". so  further we can analyze the Bike rentals with respect to year months and days.

dataset['Dates'] = dataset['Dates'].astype('datetime64[ns]')
dataset['Year'] = dataset['Dates'].dt.year
dataset['Month'] = dataset['Dates'].dt.month

dataset.head()

,Dates,Hour,Weekdays_or_weekend,Rented Bike Count,Year,Month
0,2022-12-19,0,1,2,2022,12
1,2022-12-19,1,1,1,2022,12
2,2022-12-19,3,1,1,2022,12
3,2022-12-19,6,1,1,2022,12
4,2022-12-19,7,1,3,2022,12


We're going to add the Seansons and School Holiday columns, which will show the seasons by date and the working days (school and work days) respectively.

In [34]:
def determine_saison(dates):
    year = dates.year
    if dates >= pd.Timestamp(year=year, month=3, day=20) and dates < pd.Timestamp(year=year, month=6, day=21):
        return 'Printemps'
    elif dates >= pd.Timestamp(year=year, month=6, day=21) and dates < pd.Timestamp(year=year, month=9, day=23):
        return 'Été'
    elif dates >= pd.Timestamp(year=year, month=9, day=23) and dates < pd.Timestamp(year=year, month=12, day=21):
        return 'Automne'
    else:
        return 'Hiver'
"""
# Fonction pour vérifier si une date est dans une plage de vacances
def est_vacances_scolaires(date):
    vacances_scolaires = [
        ((10, 21), (11, 5)),
        ((12, 21), (1, 7)),
        ((2, 8), (2,25)),
        ((4, 6), (4,21)),
        ((5,8), (5, 12)),
        ((7, 6), (9, 4))
    ]
    for (start_month, start_day), (end_month, end_day) in vacances_scolaires:
        start_date = datetime(date.year, start_month, start_day)
        end_date = datetime(date.year, end_month, end_day)
        
        # Ajuster l'année de fin si nécessaire pour les périodes qui traversent l'année
        if end_month < start_month:
            end_date = datetime(date.year + 1, end_month, end_day)
        
        if start_date <= date <= end_date:
            return 'School Holiday'
    return 'School day'
"""


# Fonction pour marquer les jours fériés comme vacances 
def no_work_day(dates):
    public_holiday = [
        (1, 1), (4, 1), (5, 1), (5, 8), (5, 9), (5, 20),
        (7, 14), (8, 15), (11, 1), (11, 11), (12, 25)
    ]
    return 'Public Holiday' if (dates.month, dates.day) in public_holiday else 'No Holiday'

""""
# Fonction pour marquer les jours d'événement
def event_day(dates):
    event_days = [
        (1, 1), (4, 1), (5, 1), (5, 8), (5, 9), (5, 20),
        (13, 8, 2023), (27, 8, 2023), (17, 9, 2023), (8, 10, 2023), (29, 10, 2023), (5, 11, 2023),
        (3, 12, ,2023), (14, 1, 2024), (28, 1, 2024), ,(11, 2, 2024), (25, 2, 2024), (10, 3, 2024),
        (31, 3, 2024), (14, 4, 2024), (28, 4, 2024), (11, 5, 2024)
    ]
    return 'Public Holiday' if (dates.month, dates.day) in public_holiday else 'No Holiday'
"""

# Ajouter les colonnes 'Seasons' et 'School_Holiday'
dataset['Seasons'] = dataset['Dates'].apply(determine_saison)
dataset['Holiday'] = dataset['Dates'].apply(no_work_day)

# Marquer également les weekends comme des vacances 
dataset.loc[dataset['Weekdays_or_weekend'] == 0, 'Holiday'] = 'Public Holiday'

dataset.tail()

,Dates,Hour,Weekdays_or_weekend,Rented Bike Count,Year,Month,Seasons,Holiday
14078,2024-08-11,19,0,880,2024,8,Été,Public Holiday
14079,2024-08-11,20,0,890,2024,8,Été,Public Holiday
14080,2024-08-11,21,0,724,2024,8,Été,Public Holiday
14081,2024-08-11,22,0,592,2024,8,Été,Public Holiday
14082,2024-08-11,23,0,559,2024,8,Été,Public Holiday


In [35]:
dataset = dataset.drop(columns=['Weekdays_or_weekend'])
dataset.head()

,Dates,Hour,Rented Bike Count,Year,Month,Seasons,Holiday
0,2022-12-19,0,2,2022,12,Automne,No Holiday
1,2022-12-19,1,1,2022,12,Automne,No Holiday
2,2022-12-19,3,1,2022,12,Automne,No Holiday
3,2022-12-19,6,1,2022,12,Automne,No Holiday
4,2022-12-19,7,3,2022,12,Automne,No Holiday


In [36]:
weather_set = pd.read_csv('C:/Users/39812/OneDrive - Aix Marseille Provence Métropole/Documents/Bike Project/Datasets/H_13_latest-2023-2024.csv.gz', compression ='gzip', sep = ';')
weather_set.tail(10)

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJHH,RR1,QRR1,DRR1,QDRR1,...,INS,QINS,INS2,QINS2,TLAGON,QTLAGON,TVEGETAUX,QTVEGETAUX,ECOULEMENT,QECOULEMENT
299915,13111002,VAUVENARGUES,43.551,5.68,565,2024081718,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299916,13111002,VAUVENARGUES,43.551,5.68,565,2024081719,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299917,13111002,VAUVENARGUES,43.551,5.68,565,2024081720,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299918,13111002,VAUVENARGUES,43.551,5.68,565,2024081721,0.0,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299919,13111002,VAUVENARGUES,43.551,5.68,565,2024081722,0.0,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299920,13111002,VAUVENARGUES,43.551,5.68,565,2024081723,0.0,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299921,13111002,VAUVENARGUES,43.551,5.68,565,2024081800,0.0,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299922,13111002,VAUVENARGUES,43.551,5.68,565,2024081801,0.0,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299923,13111002,VAUVENARGUES,43.551,5.68,565,2024081802,0.0,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299924,13111002,VAUVENARGUES,43.551,5.68,565,2024081803,0.0,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## This is the second set of data presenting the weather in Marseille based on several meteorological factors

We have a CSV file with descriptions of the various fields in the datagame, so we won't bother renaming the columns here.

In [37]:
#consultation of the variable presenting the different cities in the dataset
weather_set['NOM_USUEL'].value_counts()

NOM_USUEL
AIX EN PROVENCE          14284
ARLES                    14284
TRETS                    14284
TARASCON                 14284
SALON DE PROVENCE        14284
ST CHAMAS                14284
ST CANNAT                14284
PEYROLLES EN PROVENCE    14284
MIMET                    14284
MARSEILLE                14284
MARSEILLE-OBS            14284
MARIGNANE                14284
ISTRES                   14284
EYRAGUES                 14284
LA DESTROUSSE_SAPC       14284
BEC DE L AIGLE           14284
CASSIS                   14284
AUBAGNE                  14284
VAUVENARGUES             14284
CAP COURONNE             14276
CUGES-LES-PINS           14253
Name: count, dtype: int64

In [38]:
#Selection of Marseille and Marignane because the 2 cities have very similar weather conditions.

weather_Mar = weather_set[weather_set['NOM_USUEL'].isin(['MARSEILLE', 'MARIGNANE'])]
weather_Mar.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJHH,RR1,QRR1,DRR1,QDRR1,...,INS,QINS,INS2,QINS2,TLAGON,QTLAGON,TVEGETAUX,QTVEGETAUX,ECOULEMENT,QECOULEMENT
128525,13054001,MARIGNANE,43.437667,5.216,9,2023010100,0.0,1.0,0.0,9.0,...,0.0,9.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
128526,13054001,MARIGNANE,43.437667,5.216,9,2023010101,0.0,1.0,0.0,9.0,...,0.0,9.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
128527,13054001,MARIGNANE,43.437667,5.216,9,2023010102,0.0,1.0,0.0,9.0,...,0.0,9.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
128528,13054001,MARIGNANE,43.437667,5.216,9,2023010103,0.0,1.0,0.0,9.0,...,0.0,9.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
128529,13054001,MARIGNANE,43.437667,5.216,9,2023010104,0.0,1.0,0.0,9.0,...,0.0,9.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN


Nous allons corriger les champs manquants des colonnes GLO 'rayonnement global du soleil' et VV 'visibilité' de Marseille par les similitudes disponibles de Marignane

In [39]:
# Convertir la colonne AAAAMMJJHH en format datetime en copiant explicitement le DataFrame
weather_Mar = weather_Mar.copy()
weather_Mar['Dates'] = pd.to_datetime(weather_Mar['AAAAMMJJHH'], format='%Y%m%d%H')

#Suppression AAAAMMJJHH
weather_Mar = weather_Mar.drop(columns=['AAAAMMJJHH'])

# Séparer les données pour Marseille et Marignane
marseille_data = weather_Mar[weather_Mar['NOM_USUEL'] == 'MARSEILLE'].copy()
marignane_data = weather_Mar[weather_Mar['NOM_USUEL'] == 'MARIGNANE'].copy()

# Aligner les dates
marignane_data_aligned = marignane_data.set_index('Dates').reindex(marseille_data['Dates']).reset_index()

# Remplacer les colonnes 'VV' et 'GLO' de Marseille par celles de Marignane en utilisant .loc
marseille_data.loc[:, 'VV'] = marignane_data_aligned['VV'].values
marseille_data.loc[:, 'GLO'] = marignane_data_aligned['GLO'].values

# Fusionner les données modifiées de Marseille avec les données originales
weather_Mar_updated = pd.concat([weather_Mar[weather_Mar['NOM_USUEL'] != 'MARSEILLE'], marseille_data])
weather_Mar_updated = weather_Mar_updated[weather_Mar_updated['NOM_USUEL'].isin(['MARSEILLE'])]
weather_Mar_updated.tail()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,RR1,QRR1,DRR1,QDRR1,FF,...,QINS,INS2,QINS2,TLAGON,QTLAGON,TVEGETAUX,QTVEGETAUX,ECOULEMENT,QECOULEMENT,Dates
171372,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,NaN,NaN,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-17 23:00:00
171373,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-18 00:00:00
171374,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,NaN,NaN,0.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-18 01:00:00
171375,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,NaN,NaN,1.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-18 02:00:00
171376,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,NaN,NaN,2.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-18 03:00:00


In [40]:
weather_Mar_updated = weather_Mar_updated.reset_index(drop=True)
#observer les valeurs manquantes de VV
weather_Mar_updated.loc[weather_Mar_updated['VV'].isnull()]

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,RR1,QRR1,DRR1,QDRR1,FF,...,QINS,INS2,QINS2,TLAGON,QTLAGON,TVEGETAUX,QTVEGETAUX,ECOULEMENT,QECOULEMENT,Dates
7747,13055029,MARSEILLE,43.310667,5.479167,192,0.0,1.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-19 19:00:00


In [41]:
#Remplacer ces valeurs manquantes par les valeurs réelles observées 
weather_Mar_updated['VV'] = weather_Mar_updated['VV'].fillna(11060)

In [42]:
#observer les valeurs manquantes de DHUMI80  
weather_Mar_updated.loc[weather_Mar_updated['DHUMI80'].isnull()]

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,RR1,QRR1,DRR1,QDRR1,FF,...,QINS,INS2,QINS2,TLAGON,QTLAGON,TVEGETAUX,QTVEGETAUX,ECOULEMENT,QECOULEMENT,Dates
7258,13055029,MARSEILLE,43.310667,5.479167,192,0.2,1.0,NaN,NaN,4.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-10-30 10:00:00


In [43]:
weather_Mar_updated.loc[7257, 'DHUMI80'] = 50
weather_Mar_updated.loc[7258, 'DHUMI80'] = 60

In [44]:
#observer les valeurs manquantes de DD
weather_Mar_updated.loc[weather_Mar_updated['DD'].isnull()]

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,RR1,QRR1,DRR1,QDRR1,FF,...,QINS,INS2,QINS2,TLAGON,QTLAGON,TVEGETAUX,QTVEGETAUX,ECOULEMENT,QECOULEMENT,Dates
3224,13055029,MARSEILLE,43.310667,5.479167,192,0.0,1.0,NaN,NaN,1.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-15 08:00:00
3225,13055029,MARSEILLE,43.310667,5.479167,192,0.0,1.0,NaN,NaN,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-15 09:00:00
3226,13055029,MARSEILLE,43.310667,5.479167,192,0.0,1.0,NaN,NaN,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-15 10:00:00


In [45]:
#Remplacer ces valeurs manquantes par les valeurs réelles observées 
weather_Mar_updated.loc[3224, 'DD'] = 310
weather_Mar_updated.loc[3225, 'DD'] = 300
weather_Mar_updated.loc[3226, 'DD'] = 290

In [46]:
weather_Mar_updated.isnull().sum()

NUM_POSTE          0
NOM_USUEL          0
LAT                0
LON                0
ALTI               0
               ...  
TVEGETAUX      14284
QTVEGETAUX     14284
ECOULEMENT     14284
QECOULEMENT    14284
Dates              0
Length: 204, dtype: int64

Nous procédons maintenant à la suppresion des facteurs métérologiques non fournis (champs vides)

In [47]:
# Calcul du nombre et du pourcentage de valeurs manquantes
missing_values = weather_Mar_updated.isnull().sum()
missing_percentage = (missing_values / len(weather_Mar_updated)) * 100

# Créer un DataFrame pour afficher les valeurs manquantes et leur pourcentage
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})

# Trier les colonnes par le nombre de valeurs manquantes en ordre ascendant
missing_data_sorted = missing_data.sort_values(by='Missing Values', ascending=True)

# Afficher les colonnes avec des valeurs manquantes triées par ordre ascendant
print("Valeurs manquantes par colonne (ordre ascendant) :")
print(missing_data_sorted[missing_data_sorted['Missing Values'] >= 0])

Valeurs manquantes par colonne (ordre ascendant) :
           Missing Values  Percentage
NUM_POSTE               0         0.0
TD                      0         0.0
QTD                     0         0.0
TN                      0         0.0
QTN                     0         0.0
...                   ...         ...
QN1                 14284       100.0
C1                  14284       100.0
QC1                 14284       100.0
QVV                 14284       100.0
CM                  14284       100.0

[204 rows x 2 columns]


In [48]:
# Identifier les colonnes avec plus de 1% de valeurs manquantes
columns_to_drop = missing_data[missing_data['Percentage'] > 1].index
columns_to_drop

Index(['DRR1', 'QDRR1', 'FF2', 'QFF2', 'DD2', 'QDD2', 'FXI2', 'QFXI2', 'DXI2',
       'QDXI2',
       ...
       'INS', 'QINS', 'INS2', 'QINS2', 'TLAGON', 'QTLAGON', 'TVEGETAUX',
       'QTVEGETAUX', 'ECOULEMENT', 'QECOULEMENT'],
      dtype='object', length=148)

In [49]:
# Suppression des colonnes avec plusiuers valeurs manquantes
weather_Mar_cleaned = weather_Mar_updated.drop(columns=columns_to_drop)
weather_Mar_cleaned = weather_Mar_cleaned.reset_index(drop=True)
weather_Mar_cleaned.tail(5)

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,RR1,QRR1,FF,QFF,DD,...,QHUX,DHUMI40,QDHUMI40,DHUMI80,QDHUMI80,TSV,QTSV,VV,GLO,Dates
14279,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,0.7,9.0,330.0,...,9.0,0.0,9.0,60.0,9.0,22.4,9.0,48600.0,0.0,2024-08-17 23:00:00
14280,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,0.0,9.0,0.0,...,9.0,0.0,9.0,28.0,9.0,22.0,9.0,49820.0,0.0,2024-08-18 00:00:00
14281,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,0.9,9.0,280.0,...,9.0,0.0,9.0,60.0,9.0,20.7,9.0,42560.0,0.0,2024-08-18 01:00:00
14282,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,1.2,9.0,280.0,...,9.0,0.0,9.0,60.0,9.0,20.7,9.0,46410.0,0.0,2024-08-18 02:00:00
14283,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,2.4,9.0,270.0,...,9.0,0.0,9.0,60.0,9.0,20.6,9.0,57400.0,0.0,2024-08-18 03:00:00


In [50]:
# Vérifictaion présence de valeurs manquantes
weather_Mar_cleaned.isnull().sum()

NUM_POSTE    0
NOM_USUEL    0
LAT          0
LON          0
ALTI         0
RR1          0
QRR1         0
FF           0
QFF          0
DD           0
QDD          3
FXY          4
QFXY         4
DXY          4
QDXY         4
HXY          4
QHXY         4
FXI          0
QFXI         0
DXI          4
QDXI         4
HXI          4
QHXI         4
T            0
QT           0
TD           0
QTD          0
TN           0
QTN          0
HTN          0
QHTN         0
TX           2
QTX          0
HTX          2
QHTX         0
DG           3
QDG          3
U            0
QU           0
UN           0
QUN          0
HUN          1
QHUN         1
UX           0
QUX          0
HUX          1
QHUX         1
DHUMI40      1
QDHUMI40     1
DHUMI80      0
QDHUMI80     1
TSV          0
QTSV         0
VV           0
GLO          0
Dates        0
dtype: int64

In [51]:
#Extraction de colonnes 'Year', 'Month', 'Day', 'hour'
weather_Mar_cleaned['Dates'] = weather_Mar_cleaned['Dates'].astype('datetime64[ns]')
weather_Mar_cleaned['Year'] = weather_Mar_cleaned['Dates'].dt.year
weather_Mar_cleaned['Month'] = weather_Mar_cleaned['Dates'].dt.month
weather_Mar_cleaned['Hour'] = weather_Mar_cleaned['Dates'].dt.hour
weather_Mar_cleaned['Day'] = weather_Mar_cleaned['Dates'].dt.day
weather_Mar_cleaned['Dates'] = weather_Mar_cleaned['Dates'].dt.date
weather_Mar_cleaned.tail()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,RR1,QRR1,FF,QFF,DD,...,QDHUMI80,TSV,QTSV,VV,GLO,Dates,Year,Month,Hour,Day
14279,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,0.7,9.0,330.0,...,9.0,22.4,9.0,48600.0,0.0,2024-08-17,2024,8,23,17
14280,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,0.0,9.0,0.0,...,9.0,22.0,9.0,49820.0,0.0,2024-08-18,2024,8,0,18
14281,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,0.9,9.0,280.0,...,9.0,20.7,9.0,42560.0,0.0,2024-08-18,2024,8,1,18
14282,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,1.2,9.0,280.0,...,9.0,20.7,9.0,46410.0,0.0,2024-08-18,2024,8,2,18
14283,13055029,MARSEILLE,43.310667,5.479167,192,0.0,9.0,2.4,9.0,270.0,...,9.0,20.6,9.0,57400.0,0.0,2024-08-18,2024,8,3,18


In [52]:
weather_Mar_cleaned.Dates = pd.to_datetime(weather_Mar_cleaned.Dates)
# Extraire le nom du jour de la semaine
weather_Mar_cleaned['weekday'] = weather_Mar_cleaned['Dates'].dt.day_name()

# Vérifier le résultat
print(weather_Mar_cleaned[['Dates', 'weekday']].tail())

           Dates   weekday
14279 2024-08-17  Saturday
14280 2024-08-18    Sunday
14281 2024-08-18    Sunday
14282 2024-08-18    Sunday
14283 2024-08-18    Sunday


In [53]:
#Suppresion des varaibles en doublon (exprimant le meme carcatere metérologique)
weather_Mar_final = weather_Mar_cleaned.drop(columns=['QRR1', 'QFF', 'QDD', 'FXY', 'QFXY', 'DXY', 'QDXY', 'HXY', 'QHXY', 'QFXI', 'DXI', 'QDXI', 'HXI', 'QHXI', 'DXI', 'QDXI', 'HXI', 'QHXI', 'QT', 'QTD', 'TN', 'QTN', 'HTN', 'QHTN', 'TX', 'QTX', 'HTX', 'QHTX', 'DG', 'QDG', 'QU', 'UN', 'QUN', 'HUN', 'QHUN', 'UX', 'QUX', 'HUX', 'QHUX', 'DHUMI40', 'QDHUMI40', 'QDHUMI80', 'QTSV'])

#Suppresion des variables (métérologiques) jugées non pertinentes
weather_Mar_final = weather_Mar_final.drop(columns=['LAT', 'LON', 'ALTI', 'NUM_POSTE', 'NOM_USUEL'])
weather_Mar_final.head()

,RR1,FF,DD,FXI,T,TD,U,DHUMI80,TSV,VV,GLO,Dates,Year,Month,Hour,Day,weekday
0,0.0,3.8,100.0,10.5,13.4,10.8,84.0,60.0,12.9,34060.0,0.0,2023-01-01,2023,1,0,1,Sunday
1,0.0,3.8,80.0,9.4,13.6,10.8,83.0,60.0,12.9,39322.0,0.0,2023-01-01,2023,1,1,1,Sunday
2,0.0,3.8,100.0,8.1,13.9,10.6,81.0,60.0,12.8,39468.0,0.0,2023-01-01,2023,1,2,1,Sunday
3,0.0,5.0,90.0,10.8,14.0,10.8,81.0,60.0,12.9,38313.0,0.0,2023-01-01,2023,1,3,1,Sunday
4,0.0,5.3,100.0,11.7,14.4,11.2,81.0,60.0,13.3,41397.0,0.0,2023-01-01,2023,1,4,1,Sunday


In [54]:
dataset.tail(10)

,Dates,Hour,Rented Bike Count,Year,Month,Seasons,Holiday
14073,2024-08-11,14,513,2024,8,Été,Public Holiday
14074,2024-08-11,15,570,2024,8,Été,Public Holiday
14075,2024-08-11,16,630,2024,8,Été,Public Holiday
14076,2024-08-11,17,774,2024,8,Été,Public Holiday
14077,2024-08-11,18,897,2024,8,Été,Public Holiday
14078,2024-08-11,19,880,2024,8,Été,Public Holiday
14079,2024-08-11,20,890,2024,8,Été,Public Holiday
14080,2024-08-11,21,724,2024,8,Été,Public Holiday
14081,2024-08-11,22,592,2024,8,Été,Public Holiday
14082,2024-08-11,23,559,2024,8,Été,Public Holiday


In [55]:
weather_Mar_final['Dates'] = pd.to_datetime(weather_Mar_final['Dates'])
dataset['Dates'] = pd.to_datetime(dataset['Dates'])

# Fusionner les deux DataFrames sur les colonnes Dates et Hour
merged_df = pd.merge(weather_Mar_final, dataset, on=['Dates','Year', 'Month', 'Hour'], how='inner')

#Rangement colonne Dates
cols = list(merged_df.columns)
cols.insert(0, cols.pop(cols.index('Dates')))
cols.insert(1, cols.pop(cols.index('Year')))
cols.insert(2, cols.pop(cols.index('Month')))
cols.insert(3, cols.pop(cols.index('Day')))
cols.insert(4, cols.pop(cols.index('Hour')))
cols.insert(5, cols.pop(cols.index('Rented Bike Count')))
bike_df = merged_df[cols]

# Afficher les premières lignes du DataFrame fusionné
bike_df.tail()

,Dates,Year,Month,Day,Hour,Rented Bike Count,RR1,FF,DD,FXI,T,TD,U,DHUMI80,TSV,VV,GLO,weekday,Seasons,Holiday
13774,2024-08-11,2024,8,11,19,880,0.0,0.8,130.0,2.4,27.7,22.0,71.0,0.0,26.3,28118.0,10.0,Sunday,Été,Public Holiday
13775,2024-08-11,2024,8,11,20,890,0.0,0.8,90.0,2.8,26.3,22.7,81.0,18.0,27.6,25459.0,0.0,Sunday,Été,Public Holiday
13776,2024-08-11,2024,8,11,21,724,0.0,1.0,70.0,1.7,25.6,22.5,83.0,60.0,27.2,20466.0,0.0,Sunday,Été,Public Holiday
13777,2024-08-11,2024,8,11,22,592,0.0,0.8,80.0,1.8,25.3,22.4,84.0,60.0,27.0,17390.0,0.0,Sunday,Été,Public Holiday
13778,2024-08-11,2024,8,11,23,559,0.0,2.1,70.0,2.8,24.4,19.9,76.0,52.0,23.2,16160.0,0.0,Sunday,Été,Public Holiday


In [56]:
# enregistrer le dataframe bike_df dans un fichier CSV
Dataset = r"C:\Users\39812\OneDrive - Aix Marseille Provence Métropole\Documents\Bike Project\Datasets"
output_file = os.path.join(Dataset, "Bike_df.csv")
bike_df.to_csv(output_file, index=False)